<a href="https://colab.research.google.com/github/IvanRavarotto/Precios_Casas/blob/main/Precio_Casas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción de precios de casas (Supervisado - Regresión)
* Variables: metros_cuadrados, habitaciones, baños, ubicación, antigüedad
* Objetivo: precio
* Tarea: Entrenar un modelo para predecir el precio de una casa.

Trabajando en base al dataset brindado en este proyecto, debo diseñar un modelo 'supervisado' y de 'regresión' para que pueda precedir el precio de una casa que sea registrada como 'nueva'.

#Carga de dataset
### Columnas:
metros_cuadrados | habitaciones | banos | antiguedad | ubicacion | precio

In [25]:
#Importación de pandas para la carga del dataset a utilziar, en este caso desde un repositorio de github
import pandas as pd

# Se trae la información mediante el crudo (raw) desde github
df = pd.read_csv('https://raw.githubusercontent.com/IvanRavarotto/Precios_Casas/refs/heads/main/dataset_precios_casas.csv')

#Tratamiento de datos:
* Se define columna principal 'target'
* Formulario para ingreso de una nueva casa

In [29]:
Y = df['precio']
X = df.drop('precio', axis=1)
# Utilizamos la columna "precio" como "target" de busqueda para indicarle al modelo cual es nuestra variable objetivo
# El resto de columnas las dejamos en otra variable, eliminando a su vez la columna de "precio"


In [36]:
# Formulario de carga de datos de la nueva casa para precedir el precio:
metros_cuadrados = 10001 # @param {type:"number", min:10,max:10000}
habitaciones = 1 # @param {type:"number", min:1, max:100}
banios = 1 # @param {type:"number", min:1, max:100}
antiguedad = 1 # @param {type:"number", min:1, max:100}
ubicacion = "urbana" # @param ["urbana", "suburbana", "rural"]

casaNueva = {
    'metros_cuadrados': metros_cuadrados,
    'habitaciones': habitaciones,
    'banos': banios,
    'antiguedad': antiguedad,
    'precio': 0
}


In [42]:
from sklearn.ensemble import GradientBoostingRegressor


0
1000
